In [7]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [8]:
%%sql
SELECT * FROM sqlite_master
WHERE type = 'table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [9]:
%%sql
SELECT * FROM facts LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Columns dictionary:

* `name` - The name of the country.
* `area` - The total land and sea area of the country.
* `population` - The country's population.
* `population_growth`- The country's population growth as a percentage.
* `birth_rate` - The country's birth rate, or the number of births a year per 1,000 people.
* `death_rate` - The country's death rate, or the number of death a year per 1,000 people.
* `area`- The country's total area (both land and water).
* `area_land` - The country's land area in square kilometers.
* `area_water` - The country's waterarea in square kilometers.


In [10]:
%%sql

SELECT MIN(population), 
MAX(population), MIN(population_growth), 
MAX(population_growth) FROM facts

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Which countries represent the min and max populations?

In [12]:
%%sql
SELECT name, population FROM facts
GROUP BY name
HAVING population = (SELECT MIN(population) FROM facts)

Done.


name,population
Antarctica,0


In [29]:
%%sql
SELECT name, population FROM facts
GROUP BY name
HAVING population = (SELECT MAX(population) FROM facts)

Done.


name,population
World,7256490011


In [24]:
%%sql
SELECT AVG(population) avg_pop, 
       AVG(area) avg_area 
FROM facts
WHERE population != (SELECT population FROM facts 
                     WHERE name = 'World')


Done.


avg_pop,avg_area
32242666.56846473,582949.8523206752


Which countries have the highest population per area?

In [39]:
%%sql 
SELECT name, population, area, population/area pop_sq_ft FROM facts
GROUP BY name
HAVING population > (SELECT AVG(population) FROM facts
                     WHERE name != 'World')
    AND area < (SELECT AVG(area) FROM facts
                WHERE name != 'World')
ORDER BY pop_sq_ft DESC

Done.


name,population,area,pop_sq_ft
Bangladesh,168957745,148460,1138
"Korea, South",49115196,99720,492
Philippines,100998376,300000,336
Japan,126919659,377915,335
Vietnam,94348835,331210,284
United Kingdom,64088222,243610,263
Germany,80854408,357022,226
Italy,61855120,301340,205
Uganda,37101745,241038,153
Thailand,67976405,513120,132


Which countries have the highest water to land ratio?

In [48]:
%%sql
SELECT name, ROUND(CAST(area_water AS float)/CAST(area_land AS float),2) ratio 
FROM facts
ORDER BY ratio DESC
LIMIT 10

Done.


name,ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


Which countries' death rate is higher than its birth rate?

In [53]:
%%sql
SELECT name, death_rate, birth_rate, 
    ROUND(death_rate/birth_rate, 2) ratio FROM facts
WHERE death_rate > birth_rate
ORDER BY ratio DESC

Done.


name,death_rate,birth_rate,ratio
Bulgaria,14.44,8.92,1.62
Serbia,13.66,9.08,1.5
Latvia,14.31,10.0,1.43
Lithuania,14.27,10.1,1.41
Hungary,12.73,9.16,1.39
Monaco,9.24,6.65,1.39
Germany,11.42,8.47,1.35
Slovenia,11.37,8.42,1.35
Ukraine,14.46,10.72,1.35
Saint Pierre and Miquelon,9.72,7.42,1.31
